View Flash Match data stored in PetaStorm PySpark Format

In [90]:
import ROOT as rt
import numpy as np
from larcv import larcv
from larflow import larflow
import chart_studio as cs
import chart_studio.plotly as py
import plotly.graph_objects as go
from ctypes import c_int
larcv.load_pyutil()
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
# library with functions for plotting data
import lardly

# load utility to draw TPC outline
from lardly import DetectorOutline 
detdata = DetectorOutline()
detlines = detdata.getlines(color=(10,10,10))

# utility to plot opdet data
from lardly.ubdl.ubplot_opdet import make_opdet_plot

# PARTICLE LABEL COLORS
# from larcv/core/DataFormat/DataFormatTypes.h
#     kROIUnknown=0, ///< LArbys
#     kROICosmic,    ///< Cosmics
#     kROIBNB,       ///< BNB
#     kROIEminus,    ///< Electron
#     kROIGamma,     ///< Gamma
#     kROIPizero,    ///< Pi0
#     kROIMuminus,   ///< Muon
#     kROIKminus,    ///< Kaon
#     kROIPiminus,   ///< Charged Pion
#     kROIProton,    ///< Proton
#     kROITypeMax    ///< enum element counter
ssnetcolor = {0:np.array((0,0,0)),     # kROIUnknown                                                                                                                                                   
              1:np.array((255,0,0)),   # kROICosmic (not used)                                                                                                                                       
              2:np.array((0,255,0)),   # kROIBNB (not used)                                                                                                                             
              3:np.array((0,0,255)),   # kROIEminus (e-/e+)                                                                                                                                              
              4:np.array((255,0,255)), # kROIGamma                                                                                                                                                 
              5:np.array((0,255,255)), # kROIPizero                                                                                                                                            
              6:np.array((255,255,0)), # kROImuminus (mu-/mu+)
              7:np.array((123,300,10)),# kROIKminus (k+/k-)
              8:np.array((204,204,255)), # kROIPiminus (pi+/pi-)
              9:np.array((255, 165, 0))} # kProton

kpcolors = {0:np.array((255,0,0)), # nu
            1:np.array((0,255,0)), # track-start
            2:np.array((0,0,255)), # track-end
            3:np.array((255,255,0)), # shower
            4:np.array((0,255,255)), # michel
            5:np.array((255,0,255))} # delta


In [92]:
# LOAD PETASTORM PYTORCH DATALOADER INTERFACE
import flashmatchnet
from flashmatchnet.data.reader import make_dataloader

dataset_folder = 'file:///home/twongjirad/working/larbys/gen2/flashmatchdata_petastorm/temp/'

loader = make_dataloader( dataset_folder, num_epochs=None, shuffle_rows=False, batch_size=1 )
loader_iter = iter(loader)


In [95]:

LOAD_NEW_EVENT=True

if LOAD_NEW_EVENT:
    batch = next(loader_iter)

print(batch.keys())
print('Event: ',batch['event'])
print('Batch: ',batch['matchindex'])

use_opdet_index=True
use_v4_geom=True

flashpe = batch['flashpe'].squeeze()
print("Flash PE sum: ",flashpe.sum())

coord = batch['coord'][:,1:]
feat  = batch['feat']
featz = feat[:,2]
print("coord shape: ",coord.shape)
print("coord min and max: ",coord[:,0].min()," ",coord[:,0].max())
#print(feat)

# convert to positions
voxel_size_cm = 5.0
pos = coord*voxel_size_cm
pos[:,1] -= 120.0

print(pos[:,0].min()," ",pos[:,0].max())
print(pos[:,1].min()," ",pos[:,1].max())
print(pos[:,2].min()," ",pos[:,2].max())

opdet_traces = make_opdet_plot( flashpe, use_opdet_index=use_opdet_index, use_v4_geom=use_v4_geom )
detlines = detdata.getlines(color=(10,10,10))
plot_traces = detlines + opdet_traces

# PLOT CHARGE

plot_q = {
    "type":"scatter3d",
    "x": pos[:,0],
    "y": pos[:,1],
    "z": pos[:,2],
    "mode":"markers",
    "name":"Charge",
    "marker":{"color":featz,"size":2,"opacity":0.5,'colorscale':'Viridis'},
}
plot_traces += [plot_q]

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='Particle Instance Labels',
    autosize=False,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plot_traces, layout=layout)
fig.show()


dict_keys(['coord', 'feat', 'flashpe', 'event', 'matchindex', 'batchentries', 'batchstart', 'batchend'])
Event:  [0]
Batch:  [1]
Flash PE sum:  tensor(3.3380)
coord shape:  torch.Size([88, 3])
coord min and max:  tensor(0, dtype=torch.int32)   tensor(11, dtype=torch.int32)
tensor(0.)   tensor(55.)
tensor(-100.)   tensor(95.)
tensor(10.)   tensor(100.)


In [94]:
#batch = next(loader_iter)
print(batch.keys())
print('Event: ',batch['event'])
print('Batch: ',batch['matchindex'])

use_opdet_index=False
use_v4_geom=False

flashpe = batch['flashpe'].squeeze()
print("Flash PE sum: ",flashpe.sum())

coord = batch['coord'][:,1:]
feat  = batch['feat']
featz = feat[:,2]
print("coord shape: ",coord.shape)
print("coord min and max: ",coord[:,0].min()," ",coord[:,0].max())
#print(feat)

# convert to positions
voxel_size_cm = 5.0
pos = coord*voxel_size_cm
pos[:,1] -= 120.0

print(pos[:,0].min()," ",pos[:,0].max())
print(pos[:,1].min()," ",pos[:,1].max())
print(pos[:,2].min()," ",pos[:,2].max())

opdet_traces = make_opdet_plot( flashpe, use_opdet_index=use_opdet_index, use_v4_geom=use_v4_geom )
detlines = detdata.getlines(color=(10,10,10))
plot_traces = detlines + opdet_traces

# PLOT CHARGE

plot_q = {
    "type":"scatter3d",
    "x": pos[:,0],
    "y": pos[:,1],
    "z": pos[:,2],
    "mode":"markers",
    "name":"Charge",
    "marker":{"color":featz,"size":2,"opacity":0.5,'colorscale':'Viridis'},
}
plot_traces += [plot_q]

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='Particle Instance Labels',
    autosize=False,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plot_traces, layout=layout)
fig.show()

dict_keys(['coord', 'feat', 'flashpe', 'event', 'matchindex', 'batchentries', 'batchstart', 'batchend'])
Event:  [0]
Batch:  [0]
Flash PE sum:  tensor(0.3150)
coord shape:  torch.Size([76, 3])
coord min and max:  tensor(35, dtype=torch.int32)   tensor(42, dtype=torch.int32)
tensor(175.)   tensor(210.)
tensor(-110.)   tensor(105.)
tensor(795.)   tensor(930.)
Set OpCh[ 29 ] to OpDet[ 0 ]
Set OpCh[ 23 ] to OpDet[ 10 ]
Set OpCh[ 19 ] to OpDet[ 11 ]
Set OpCh[ 21 ] to OpDet[ 12 ]
Set OpCh[ 16 ] to OpDet[ 13 ]
Set OpCh[ 14 ] to OpDet[ 14 ]
Set OpCh[ 18 ] to OpDet[ 15 ]
Set OpCh[ 17 ] to OpDet[ 16 ]
Set OpCh[ 13 ] to OpDet[ 17 ]
Set OpCh[ 15 ] to OpDet[ 18 ]
Set OpCh[ 10 ] to OpDet[ 19 ]
Set OpCh[ 27 ] to OpDet[ 1 ]
Set OpCh[ 12 ] to OpDet[ 20 ]
Set OpCh[ 8 ] to OpDet[ 21 ]
Set OpCh[ 7 ] to OpDet[ 22 ]
Set OpCh[ 11 ] to OpDet[ 23 ]
Set OpCh[ 9 ] to OpDet[ 24 ]
Set OpCh[ 3 ] to OpDet[ 25 ]
Set OpCh[ 1 ] to OpDet[ 26 ]
Set OpCh[ 6 ] to OpDet[ 27 ]
Set OpCh[ 5 ] to OpDet[ 28 ]
Set OpCh[ 0 ] to Op

In [142]:
batch.keys()

dict_keys(['coord', 'feat', 'flashpe', 'event', 'matchindex', 'batchentries', 'batchstart', 'batchend'])